In [14]:
from gensim.models import fasttext as ft
import numpy as np
import os
import nltk
import operator
import re
import nltk
import numpy
from dataclasses import dataclass

In [19]:
class BlankSort(BlankSort):
    def __init__(self):
        self._loadData()
    def _loadData(self):
        nltk.download('wordnet')
        nltk.download('stopwords')
        if "model" not in dir(self):
            # https://fasttext.cc/docs/en/crawl-vectors.html
            type(self)._model=ft.load_facebook_vectors(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))
        type(self)._window_size=3
        type(self)._lemmatizer=nltk.wordnet.WordNetLemmatizer()
        type(self)._porter=nltk.stem.porter.PorterStemmer()
        # https://github.com/Alir3z4/stop-words
        type(self)._stops=set(line.strip() for line in open(os.path.join(os.getcwd(),"binaries/data/stopwords-en.txt"),encoding='utf8'))

In [20]:
class BlankSort(BlankSort):
    def rank(self):
        return []

In [17]:
blanksort=BlankSort()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
blanksort.rank()

[]

In [ ]:
"""
def rank(text,top=None,collapse=False):
    text=text.lower()
    tokens = nltk.word_tokenize(text)
    stemmed_words = [stemmer.lemmatize(word) for word in tokens]
    cleaned_words= [word for word in stemmed_words if porterStemmer.stem(word) not in stops and word not in stops]
    filtered_words=[]
    counts=dict()
    scores=dict()
    for word in cleaned_words:
        filtered=re.sub('[^a-zA-Z]', '', word)
        if(len(filtered)>=3 or filtered in model.vocab):
            filtered_words.append(filtered)
            counts.setdefault(filtered,0)
            counts[filtered]+=1
    global similarityMatrix
    similarityMatrix=numpy.full((len(filtered_words),len(filtered_words)),None)
    scoreList=np.zeros(len(filtered_words))
    for i in range(len(filtered_words)):
        leftBound=max(0,i-window_size)
        rightBound=min(len(filtered_words)-1,i+window_size)
        contextSize=rightBound-leftBound+1
        for j in range(i+1,rightBound+1):
            similarity_score=0.0
            if similarityMatrix[i][j]!=None:
                similarity_score=similarityMatrix[i][j]
            else:
                similarity_score=model.similarity(filtered_words[i],filtered_words[j])
                similarity_score=(similarity_score+1)/2.0
            scoreList[i]+=similarity_score
            scoreList[j]+=similarity_score
            similarityMatrix[i][j]=similarity_score
            similarityMatrix[j][i]=similarity_score
        wordScore=scoreList[i]/(counts[filtered_words[i]]*contextSize)
        if filtered_words[i] not in scores:
            scores[filtered_words[i]]=wordScore
        else:
            scores[filtered_words[i]]=min(scores[filtered_words[i]],wordScore)
    wordScores=list(map(list, scores.items()))
    wordScores=sorted(wordScores, key = lambda x: x[1])#,reverse=True)
    filteredWordScores=[]
    for word in wordScores:
        filtered=re.sub('[^a-zA-Z]', '', word[0])
        if(len(filtered)>2):
            filteredWordScores.append([filtered,word[1]])
    if collapse:
        if(top==None):
            top=max(5,len(filtered_words)//3)
        print("Phrase collection is disabled.")
        #print(scores)
        #print(filteredWordScores)
        #filteredWordScores=removeSimilarWords(filteredWordScores)
        # phraseScores=collapsePhrases(tokens,scores,filteredWordScores[:top],cleaned_words)
        #phraseScores=diversifyResults([x[0] for x in phraseScores])
        # return phraseScores[:top]
        return filteredWordScores[:top]
    else:
        if(top==None):
            top=len(filteredWordScores)//3
        return filteredWordScores[:top]
"""

In [0]:
"""
def removeSimilarWords(scores,simScore=0.8):
    N=len(scores)
    for i in range(N):
        for j in range(N):
            if(i!=j and model.similarity(scores[i][0],scores[j][0])>=simScore):
"""

In [0]:
# https://medium.com/tech-that-works/maximal-marginal-relevance-to-rerank-results-in-unsupervised-keyphrase-extraction-22d95015c7c5
def maximal_marginal_relevance(sentence_vector, phrases, embedding_matrix, lambda_constant=0.5, threshold_terms=10):
    """
    Return ranked phrases using MMR. Cosine similarity is used as similarity measure.
    :param sentence_vector: Query vector
    :param phrases: list of candidate phrases
    :param embedding_matrix: matrix having index as phrases and values as vector
    :param lambda_constant: 0.5 to balance diversity and accuracy. if lambda_constant is high ,      then higher accuracy. If lambda_constant is low then high diversity.
    :param threshold_terms: number of terms to include in result set
    :return: Ranked phrases with score
    """
    # todo: Use cosine similarity matrix for lookup among phrases instead of making call everytime.
    s = []
    r = sorted(phrases, key=lambda x: x[1], reverse=True)
    r = [i[0] for i in r]
    while len(r) > 0:
        score = 0
        phrase_to_add = ''
        for i in r:
            first_part = cosine_similarity([sentence_vector], [embedding_matrix.loc[i]])[0][0]
            second_part = 0
            for j in s:
                cos_sim = cosine_similarity([embedding_matrix.loc[i]], [embedding_matrix.loc[j[0]]])[0][0]
                if cos_sim > second_part:
                    second_part = cos_sim
            equation_score = lambda_constant*(first_part-(1-lambda_constant) * second_part)
            if equation_score > score:
                score = first_part - (1 - lambda_constant) * second_part
                phrase_to_add = i
        if phrase_to_add == '':
            phrase_to_add = i
        r.remove(phrase_to_add)
        s.append((phrase_to_add, score))
    return (s, s[:threshold_terms])[threshold_terms > len(s)]

In [0]:
def collapsePhrases(tokens,scores,filteredWordScores,cleaned_words,sort=False):
    dictionary=set()
    for word in cleaned_words:
        dictionary.add(word)
    phrases=dict()
    bagOfWords=set()
    for word in filteredWordScores:
        bagOfWords.add(word[0])
    phrase=""
    totalScore=0
    wordCount=0
    i=0
    def reset():
        nonlocal wordCount,phrase,totalScore
        if(wordCount>0):
            phrases[phrase[:len(phrase)-1]]=totalScore/wordCount
            totalScore=0
            wordCount=0
            phrase=""
    while i in range(len(tokens)):
        word=stemmer.lemmatize(tokens[i])
        if (word in stops or tokens[i] in stops) and phrase!="":
            j=i+1
            while j<len(tokens) and stemmer.lemmatize(tokens[j]) not in dictionary:
                j+=1
            if(j<len(tokens) and stemmer.lemmatize(tokens[j]) in bagOfWords):
                for k in range(i,j):
                    phrase+=tokens[k]+" "
                i=j-1
            else:
                reset()
        elif word in bagOfWords:
            totalScore+=scores[word]
            wordCount+=1
            phrase+=tokens[i]+" "
        else:
            reset()
        i+=1
    phraseScores=list(map(list, phrases.items()))
    if sort:
        phraseScores=sorted(phraseScores, key = lambda x: x[1])#len(x[0].split()))#,reverse=True)
    return phraseScores